<a href="https://colab.research.google.com/github/ericjenn/working-groups/blob/ericjenn-srpwg-wg1/safety-related-profile/documents/profile_opset/div/div.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

#!pip install onnx onnxruntime


In [3]:
import onnx
import onnxruntime as ort
import numpy as np

# Define input and output tensor names
input_name = "X"
sqrt_output_name = "SqrtOutput"

#-------------------------------------------------------------------------------
# 2-rank input tensor
#-------------------------------------------------------------------------------

# Create 2-rank input tensor
input_tensor = onnx.helper.make_tensor_value_info(input_name, onnx.TensorProto.FLOAT, [None, None])

# Create output tensor (final result after sqrt operation)
sqrt_output = onnx.helper.make_tensor_value_info(sqrt_output_name, onnx.TensorProto.FLOAT, [None, None])

# Define Sqrt node: Y = Sqrt(X)
sqrt_node = onnx.helper.make_node("Sqrt", [input_name], [sqrt_output_name])

# Create the ONNX graph
graph = onnx.helper.make_graph(
    nodes=[sqrt_node],
    name="Sqrt",
    inputs=[input_tensor],
    outputs=[sqrt_output]
)

# Create the ONNX model (Sqrt is available in opset 13)
model = onnx.helper.make_model(graph, opset_imports=[onnx.helper.make_opsetid("", 13)])  # Explicitly set opset to 13
onnx.checker.check_model(model)

# Save the model
onnx.save(model, "sqrt.onnx")

# Load and run the model using ONNX Runtime
session = ort.InferenceSession("sqrt.onnx")

def do_sqrt(x):
    # Run inference
    output = session.run(None, {input_name: x})

    x_f = np.array2string(x, separator=',', max_line_width=np.inf).replace('\n', '')
    o_f = np.array2string(output[0], separator=',', max_line_width=np.inf).replace('\n', '')

    # Display results
    print(f"X={x_f}")
    print(f"Result: sqrt(X)={o_f}")

np.set_printoptions(precision=None, floatmode='fixed')


#--------------------------------------------------------------------
# Nominal cases
#--------------------------------------------------------------------

# Case N1: 2-rank tensor (float32)
x = np.array([[1.0, 2.0],
              [3.0, 4.0]], dtype=np.float32)

do_sqrt(x)

#--------------------------------------------------------------------
# Non nominal cases (nan, -inf, 0, negative values)
#--------------------------------------------------------------------

v = [1.0, 0.0, -1.0, float("inf"), float("-inf"), float("nan")]

for x_val in v:
    x_np = np.array([[x_val],
                     [x_val]], dtype=np.float32)
    do_sqrt(x_np)



#--------------------------------------------------------------------
# Examples from documentation
#--------------------------------------------------------------------

print("\n## Example 1")

# Example 1
# X = [1, 2, 4]
x_example_1 = np.array([[1.0, 2.0, 4.0]], dtype=np.float32)
do_sqrt(x_example_1)

print("\n## Example 2")

# Example 2
# X =
# [[  0.25   2.25 ]
#  [  0      0.1  ]
#  [ 10     1000  ]]
x_example_2 = np.array([
    [0.25,   2.25],
    [0.0,    0.1],
    [10.0,  1000.0]
], dtype=np.float32)

do_sqrt(x_example_2)

print("\n## Example 3")

# Example 3
# X =
# [[  inf   nan   -inf -0.0]]
x_example_3 = np.array([
    [float("inf"), float("nan"), float("-inf"), -0.0]
], dtype=np.float32)

do_sqrt(x_example_3)


X=[[1.00000000,2.00000000], [3.00000000,4.00000000]]
Result: sqrt(X)=[[1.00000000,1.41421354], [1.73205078,2.00000000]]
X=[[1.00000000], [1.00000000]]
Result: sqrt(X)=[[1.00000000], [1.00000000]]
X=[[0.00000000], [0.00000000]]
Result: sqrt(X)=[[0.00000000], [0.00000000]]
X=[[-1.00000000], [-1.00000000]]
Result: sqrt(X)=[[nan], [nan]]
X=[[inf], [inf]]
Result: sqrt(X)=[[inf], [inf]]
X=[[-inf], [-inf]]
Result: sqrt(X)=[[nan], [nan]]
X=[[nan], [nan]]
Result: sqrt(X)=[[nan], [nan]]

## Example 1
X=[[1.00000000,2.00000000,4.00000000]]
Result: sqrt(X)=[[1.00000000,1.41421354,2.00000000]]

## Example 2
X=[[2.50000000e-01,2.25000000e+00], [0.00000000e+00,1.00000001e-01], [1.00000000e+01,1.00000000e+03]]
Result: sqrt(X)=[[ 0.50000000, 1.50000000], [ 0.00000000, 0.31622776], [ 3.16227770,31.62277603]]

## Example 3
X=[[        inf,        nan,       -inf,-0.00000000]]
Result: sqrt(X)=[[        inf,        nan,        nan,-0.00000000]]
